In [ ]:
from deconv_sunet import deconv_sunet
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import glob
import os
import platform
from astropy.utils.data import get_pkg_data_filename
from matplotlib import colors
import pandas as pd
from astropy.io import ascii,fits
from astropy.table import Table
import time

In [ ]:
os_check = platform.platform(terse=True)[:5]
if os_check == 'macOS':
    preamble = '/path/to/mac/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'
else:
    preamble = '/path/to/linux/'
    root2 = f'{preamble}Deconvolution/'
    root1 = f'{root2}Data/PHOTOMETRY/PHOTOM_CATS/'

final_files_bands = f'{root2}psfm_final_merge.csv'

psf_dir = f'{root1}PSFs/'    
test_dir = f'{root1}test_dir'

SUNet_path = f'path/to/SUNet'             # Path to SUNet repository
model_dir = f'{SUNet_path}checkpoints/'     # Path to SUNet model directory
model_name = 'SUNet_model_bestSSIM_ep-400_bs-16_ps-1.pth'

In [ ]:
cluster_df = pd.read_csv(final_files_bands)
clusters = cluster_df['Cluster'].unique()
remove_clust = [ 'SpARCS0034', 'SpARCS0036','SpARCS0215', 'SpARCS1047', 'SpARCS1613']
clusters = [x for x in clusters if x not in remove_clust]
irac = ['IRAC1', 'IRAC2', 'IRAC3', 'IRAC4']
cluster_df = cluster_df[~cluster_df['Band'].isin(irac) ]
plot_dir = f'{root2}Plots'
star_cut_cats_dir = f'{root2}star_cut_catalogs'
clusters

# Deconvolution function

In [ ]:
def decon(cluster, band, files, psf):
    #for i in range(len(j_file_paths)):
    
    for i in range(len(files)):
    #for i in range(1):
        file_id = files[i][-19:-10]
        hdu = fits.open(files[i])[0]
        image = np.array(hdu.data)
        image = np.expand_dims(image, axis=0)
        image = np.expand_dims(image, axis=0)
        #hst = fits.open(hst_data[i])[0]
        psf = psf_data_j / np.sum(psf_data_j)
        """    
        Convention :
        
            Input Ground-based noisy Image(s)   : 4D numpy array  (number of samples, channels, height, width).
                
        """
        
        #vlt = np.load('Data/vlt.npy')
        #vlt = np.array([np.load(file) for file in file_paths])[0]
        
        #print('Dimensions =', Ks.shape)
        #cmap = 'RdBu_r
    
    
        """
        Run the SUNet deconvolution
        Replace the input paths with the path to your cloned SUNet repository and the path to your SUNet model directory.
        """
        
        # Factor by which the PSF is oversampled with respect to the noisy image
        # Deconvolved ouput will have the same dimensions as the PSF
        sampling_factor = psfs.shape[2]//image.shape[2]     
        
        # Run the SUNet deconvolution
        deconv_result, _ = deconv_sunet(image, 
                                        psf, 
                                        sampling_factor = sampling_factor,                       # Sampling factor
                                        SUNet_path= f'{preamble}SUNet/' ,                # Path to SUNet repository
                                        model_dir = f'{SUNet_path}checkpoints/', 
                                        model_name = 'new_model_bestSSIM_ep-400_bs-16_ps-1.pth')    # Path to SUNet model directory
                                                 # Name of the pre-trained SUNet model)
        
        deconv_result.shape             # Shape = (vlt.shape[0], vlt.shape[1], vlt.shape[2]*sampling_factor, vlt.shape[3]*sampling_factor)
        hdu_decon = fits.PrimaryHDU(deconv_result)
        #hdu_decon.writeto(f'{root2}/{cluster}/{band}/deconvolved_fits/{cluster}_{band}_{file_id}_cutout_{label}.fits', overwrite=True)
        hdu_decon.writeto(f'{root2}/{cluster}/{band}/deconvolved_fits/{cluster}_{band}_{file_id}_cutout_decon.fits', overwrite=True)

def decon_test(cluster, band, files, psf):
    
       
    for i in range(len(files)):
        file_id = files[i][-19:-10]
        
        # Open the file using the context manager
        with fits.open(files[i]) as hdu_list:
            hdu = hdu_list[0]
            image = np.array(hdu.data)

            if image.shape != (32, 32):
                print(f"Skipping file {files[i]} due to incompatible shape: {image.shape}")
                continue  # Skip this file and proceed to the next
            image = np.expand_dims(image, axis=0)
            image = np.expand_dims(image, axis=0)
     
        # Factor by which the PSF is oversampled with respect to the noisy image
        # Use the last dimension of the PSF and the image to calculate sampling_factor
        sampling_factor = psf.shape[2] // image.shape[2]  # Use -1 to access the last dimension safely
        
        # Run the SUNet deconvolution
        deconv_result, _ = deconv_sunet(image, 
                                        psf, 
                                        sampling_factor=sampling_factor,           # Sampling factor
                                        SUNet_path=f'{preamble}SUNet/',            # Path to SUNet repository
                                        model_dir=f'{SUNet_path}checkpoints/', 
                                        model_name='new_model_bestSSIM_ep-400_bs-16_ps-1.pth')  # SUNet model

        # Save the deconvolved result as a new FITS file
        hdu_decon = fits.PrimaryHDU(deconv_result)
        hdu_decon.writeto(f'{root2}/{cluster}/{band}/deconvolved_fits/{cluster}_{band}_{file_id}_cutout_decon.fits', overwrite=True)

In [ ]:
# This is a snippet to change the file prefixes depending on the plaftorm
def change_to_mac_prefix(filename):
    prefix = '/home/six6ix6ix/OneDrive_brooksc99@ku.edu/Deconvolution/'
    new_prefix = '/Users/lordereinion/OneDrive - University of Kansas/Deconvolution/'
        
    if os_check == 'macOS':
        if filename.startswith(prefix):
            filename = filename.replace(prefix, new_prefix, 1)
            #print(cluster, filename)
            return filename
    else:
        filename = filename.replace(prefix," ", 1)
        return filename

# Execute Deconvolution

In [ ]:
for cluster in clusters:
    filtered_df = cluster_df[(cluster_df['Cluster'] == cluster) & (cluster_df['Band'] == 'VIMOSz')]
    for i in filtered_df.index:
        band = filtered_df['Band'][i]
        #data_file = filtered_df['Data File'][i]
        #noise_file = filtered_df['Noise File'][i]
        psf_file = [filtered_df['PSF File'][i]]
        galaxy_files_dir = f'{root2}{cluster}/{band}/data_cutouts/'
        galaxy_files = sorted(glob.glob(os.path.join(galaxy_files_dir, '*.fits')))
        #file_1 = decon_test(cluster, band, galaxy_files, psf_file)
        psf_data = np.array([fits.getdata(file, header=False) for file in psf_file])
        psf = psf_data / np.sum(psf_data)
        decon_test(cluster, band, galaxy_files, psf)